In [1]:
import math
import numpy as np
import random

In [2]:
#Sample of xor table
Training_data = [
    {'inputs': [0, 1],'target': [1]},
    {'inputs': [1, 0],'target': [1]},
    {'inputs': [0, 0],'target': [0]},
    {'inputs': [1, 1],'target': [0]}
]

In [3]:
class NeuralNetwork(object):
    #Step 1: prepare data
    def __init__(self, input_layer, hidden_layer, output_layer):
        #Prepare a simple nn with 3 layers
        self.i_nodes = input_layer
        self.h_nodes = hidden_layer
        self.o_nodes = output_layer

        #Sigmoid activation function
        self.sigmoid = np.vectorize(lambda x: 1 / (1 + math.exp(-x)))
        
        #Derivative of sigmoid
        self.dsigmoid = np.vectorize(lambda x: x * (1 - x))

        #Learning rate (size of the corrective steps)
        self.learning_rate = 0.1

        #Generate random weights of input to hidden layer connections
        self.weights_ih = np.random.uniform(-1, 1, (self.h_nodes, self.i_nodes))
        #Generate random weights of hidden to output layer connections
        self.weights_ho = np.random.uniform(-1, 1, (self.o_nodes, self.h_nodes))

        #Generate random biases of the hidden layer
        self.bias_h = np.random.rand(self.h_nodes, 1)
        #Generate random biases of the output layer
        self.bias_o = np.random.rand(self.o_nodes, 1)

    #Step 2: Apply feedforward
    def feedforward(self, input_array):

        #Transform the input array into a vector
        input_v = np.asmatrix(input_array)
        input_v = np.reshape(input_v, (self.i_nodes, 1))

        #Calculate new hidden layer results
        hidden = self.weights_ih.dot(input_v)
        hidden = hidden + self.bias_h

        #Run the hidden results through sigmoid
        hidden = self.sigmoid(hidden)

        #Calculate new output layer results
        output = self.weights_ho.dot(hidden)
        output = output + self.bias_o

        #Run the output results through sigomid
        output = self.sigmoid(output)

        return output
    
    #Step 3: Do back propagation and train data n times
    def train(self, input_array, answers_array):

        #Transform the input array into a vector
        input_v = np.asmatrix(input_array)
        input_v = np.reshape(input_v, (self.i_nodes, 1))

        #Calculate hidden layer results
        hidden = self.weights_ih.dot(input_v)
        hidden = hidden + self.bias_h
        #Run hidden result through sigmoid
        hidden = self.sigmoid(hidden)
        
        #Calculate output layer results
        output = self.weights_ho.dot(hidden)
        output = output + self.bias_o
        #Run output result through sigmoid
        output = self.sigmoid(output)

        #Transform the answer array into a vector
        answers_vector = np.asmatrix(answers_array)
        answers_vector = np.reshape(answers_vector, (self.o_nodes, 1))

        #Calculate the error using ...
        output_errors = answers_vector - output

        #Calculate output gradient
        #Formula: LR * E * d (final_output).hidden_layer_values_transposed
        
        #1. Calculate derivative of output through the derivative sigmoid function
        derivative_output = self.dsigmoid(output)
        
        #2. Multiply d and E
        weights_gradient = np.multiply(derivative_output, output_errors)
        
        #3. Multiply d and E with learning rate
        weights_gradient = weights_gradient * self.learning_rate

        hidden_t = hidden.transpose()
        weights_ho_deltas = weights_gradient.dot(hidden_t)
        
        #Adjust the weights from hidden to output
        self.weights_ho = self.weights_ho + weights_ho_deltas
        self.bias_o = self.bias_o + weights_gradient

        #Calculate the hidden layer error
        weights_ho_t = self.weights_ho.transpose()
        hidden_errors = weights_ho_t.dot(output_errors)

        #Calculate hidden gradient
        #Formula: LR * E * d (final_output).hidden_layer_values_transposed
        
        #1. Calculate derivative of hidden through the derivative sigmoid function
        derivative_output = self.dsigmoid(hidden)

        #2. Multiply d and E
        hidden_gradient = np.multiply(derivative_output, hidden_errors)
        
        #3. Multiply d and E with learning rate
        hidden_gradient = hidden_gradient * self.learning_rate
        
        inputs_t = input_v.transpose()
        weights_ih_d = hidden_gradient.dot(inputs_t)

        #Adjust the weights from input to hidden
        self.weights_ih = self.weights_ih + weights_ih_d
        self.bias_h = self.bias_h + hidden_gradient
  

if __name__ == "__main__":
    #Define 2 for input layer, 4 for hidden layer, 1 for output layer
    nn = NeuralNetwork(2, 4, 1)

    #Train the data using neural network n (20,000) times
    for i in range(20000):
        el = random.choice(Training_data)
        nn.train(el['inputs'], el['target'])
    
    #Test the model after training and print the results
    print('Test1 for 0 and 0 = {}'.format(nn.feedforward([0, 0])))
    print('Test2 for 0 and 1 = {}'.format(nn.feedforward([0, 1])))
    print('Test3 for 1 and 0 = {}'.format(nn.feedforward([1, 0])))
    print('Test4 for 1 and 1 = {}'.format(nn.feedforward([1, 1])))

Test1 for 0 and 0 = [[0.02986487]]
Test2 for 0 and 1 = [[0.96529992]]
Test3 for 1 and 0 = [[0.96328172]]
Test4 for 1 and 1 = [[0.04276897]]
